# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
!pip install gmaps
!jupyter nbextension enable --py --sys-prefix widgetsnbextension
!jupyter nbextension enable --py --sys-prefix gmaps
!jupyter nbextension enable --py gmaps
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
# Import API key
from api_keys import g_key


Config option `kernel_spec_manager_class` not recognized by `EnableNBExtensionApp`.
Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok
Config option `kernel_spec_manager_class` not recognized by `EnableNBExtensionApp`.
Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok
Config option `kernel_spec_manager_class` not recognized by `EnableNBExtensionApp`.
Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#CityData.csv
vacation_data = pd.read_csv('../WeatherPy/CityData.csv')
vacation_data.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,los llanos de aridane,28.6585,-17.9182,286.50,77,20,2.06,ES,1612743000
1,mathbaria,22.2876,89.9593,287.17,49,0,3.18,BD,1612743098
2,sidi ali,36.0998,0.4206,284.18,80,4,12.37,DZ,1612743098
3,bluff,-46.6000,168.3333,290.15,65,99,2.07,NZ,1612743038
4,pisco,-13.7000,-76.2167,295.35,83,75,5.66,PE,1612742966


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)
locations = vacation_data[['Lat', 'Lng']]
humidity = vacation_data['Humidity']

fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False,
                               max_intensity=100,point_radius = 1)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [12]:
ideal_weather = vacation_data[(vacation_data['Max Temp'] < 85) & 
                              (vacation_data['Max Temp'] > 65) &
                              (vacation_data['Wind Speed'] < 10) &
                              (vacation_data['Cloudiness'] == 0) ]
vacation_data.head()                              



,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,los llanos de aridane,28.6585,-17.9182,286.50,77,20,2.06,ES,1612743000
1,mathbaria,22.2876,89.9593,287.17,49,0,3.18,BD,1612743098
2,sidi ali,36.0998,0.4206,284.18,80,4,12.37,DZ,1612743098
3,bluff,-46.6000,168.3333,290.15,65,99,2.07,NZ,1612743038
4,pisco,-13.7000,-76.2167,295.35,83,75,5.66,PE,1612742966


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [18]:
from api_keys import weather_api_key
url = "http://api.openweathermap.org/data/2.5/weather?"
query_url = url + "appid=" + weather_api_key + "&q="
city = 'necochea'
final_url = query_url+city
final_url

'http://api.openweathermap.org/data/2.5/weather?appid=172259f7a85e648cdb9f818fa80dbc8f&q=necochea'

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
